# SQL Tutorial: Main Functionalities

Structured Query Language (SQL) is the standard language for relational database management systems. This tutorial covers the essential SQL functionalities, including creating databases, manipulating data, and querying information.

## 1. Creating and Managing Tables

In [27]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

### Create a Table

In [28]:
# Create a table
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL,
    email TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)''')
conn.commit()

## 2. Inserting Data

### Insert a Single Row

In [29]:
# Insert a single row
cursor.execute("INSERT INTO users (username, email) VALUES ('john_doe', 'john@example.com')")
conn.commit()

### Insert Multiple Rows

In [30]:
# Insert multiple rows
users = [
    ('jane_doe', 'jane@example.com'),
    ('alice_smith', 'alice@example.com')
]
cursor.executemany("INSERT INTO users (username, email) VALUES (?, ?)", users)
conn.commit()

## 3. Querying Data

### Basic Select Query

In [31]:
# Basic select query
df = pd.read_sql_query("SELECT * FROM users", conn)
df

,id,username,email,created_at
0,2,jane_doe,jane@example.com,2024-06-05 19:24:39
1,3,alice_smith,alice@example.com,2024-06-05 19:24:39
2,5,jane_doe,jane@example.com,2024-06-05 19:24:49
3,6,alice_smith,alice@example.com,2024-06-05 19:24:49
4,7,john_doe,john@example.com,2024-06-05 19:25:10
5,8,jane_doe,jane@example.com,2024-06-05 19:25:10
6,9,alice_smith,alice@example.com,2024-06-05 19:25:10


### Where Clause

In [32]:
# Where clause
df_where = pd.read_sql_query("SELECT * FROM users WHERE username = 'john_doe'", conn)
df_where

,id,username,email,created_at
0,7,john_doe,john@example.com,2024-06-05 19:25:10


### Order By Clause

In [33]:
# Order by clause
df_order = pd.read_sql_query("SELECT * FROM users ORDER BY created_at DESC", conn)
df_order

,id,username,email,created_at
0,7,john_doe,john@example.com,2024-06-05 19:25:10
1,8,jane_doe,jane@example.com,2024-06-05 19:25:10
2,9,alice_smith,alice@example.com,2024-06-05 19:25:10
3,5,jane_doe,jane@example.com,2024-06-05 19:24:49
4,6,alice_smith,alice@example.com,2024-06-05 19:24:49
5,2,jane_doe,jane@example.com,2024-06-05 19:24:39
6,3,alice_smith,alice@example.com,2024-06-05 19:24:39


## 4. Updating Data

### Update Specific Rows

In [34]:
# Update specific rows
cursor.execute("UPDATE users SET email = 'new_email@example.com' WHERE username = 'john_doe'")
conn.commit()
df_updated = pd.read_sql_query("SELECT * FROM users WHERE username = 'john_doe'", conn)
df_updated

,id,username,email,created_at
0,7,john_doe,new_email@example.com,2024-06-05 19:25:10


## 5. Deleting Data

### Delete Specific Rows

In [35]:
# Delete specific rows
cursor.execute("DELETE FROM users WHERE username = 'john_doe'")
conn.commit()
df_deleted = pd.read_sql_query("SELECT * FROM users", conn)
df_deleted

,id,username,email,created_at
0,2,jane_doe,jane@example.com,2024-06-05 19:24:39
1,3,alice_smith,alice@example.com,2024-06-05 19:24:39
2,5,jane_doe,jane@example.com,2024-06-05 19:24:49
3,6,alice_smith,alice@example.com,2024-06-05 19:24:49
4,8,jane_doe,jane@example.com,2024-06-05 19:25:10
5,9,alice_smith,alice@example.com,2024-06-05 19:25:10


## 6. Advanced Queries

### Join Tables

In [36]:
# Create another table for join example
cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    product TEXT,
    FOREIGN KEY (user_id) REFERENCES users (id)
)''')
conn.commit()

# Insert data into orders table
orders = [
    (1, 'Laptop'),
    (2, 'Smartphone'),
    (3, 'Tablet')
]
cursor.executemany("INSERT INTO orders (user_id, product) VALUES (?, ?)", orders)
conn.commit()

In [37]:
# Inner Join
df_inner_join = pd.read_sql_query('''
SELECT users.username, orders.product
FROM users
INNER JOIN orders ON users.id = orders.user_id
''', conn)
df_inner_join

,username,product
0,jane_doe,Smartphone
1,alice_smith,Tablet
2,jane_doe,Smartphone
3,alice_smith,Tablet


In [38]:
# Left Join
df_left_join = pd.read_sql_query('''
SELECT users.username, orders.product
FROM users
LEFT JOIN orders ON users.id = orders.user_id
''', conn)
df_left_join

,username,product
0,jane_doe,Smartphone
1,jane_doe,Smartphone
2,alice_smith,Tablet
3,alice_smith,Tablet
4,jane_doe,None
5,alice_smith,None
6,jane_doe,None
7,alice_smith,None


In [39]:
# Right Join
df_right_join = pd.read_sql_query('''
SELECT users.username, orders.product
FROM users
RIGHT JOIN orders ON users.id = orders.user_id
''', conn)
df_right_join

,username,product
0,jane_doe,Smartphone
1,jane_doe,Smartphone
2,alice_smith,Tablet
3,alice_smith,Tablet
4,None,Laptop
5,None,Laptop


### Group By Clause

In [40]:
# Group by clause
df_group_by = pd.read_sql_query('''
SELECT COUNT(*) as count, created_at
FROM users
GROUP BY created_at
''', conn)
df_group_by

,count,created_at
0,2,2024-06-05 19:24:39
1,2,2024-06-05 19:24:49
2,2,2024-06-05 19:25:10


### Having Clause

In [41]:
# Having clause
df_having = pd.read_sql_query('''
SELECT COUNT(*) as count, created_at
FROM users
GROUP BY created_at
HAVING COUNT(*) > 1
''', conn)
df_having

,count,created_at
0,2,2024-06-05 19:24:39
1,2,2024-06-05 19:24:49
2,2,2024-06-05 19:25:10


## 7. Views

### Create a View

In [42]:
# Create a view
cursor.execute('''
CREATE VIEW user_emails AS
SELECT username, email
FROM users
''')
conn.commit()

### Query a View

In [43]:
# Query a view
df_view = pd.read_sql_query("SELECT * FROM user_emails", conn)
df_view

,username,email
0,jane_doe,jane@example.com
1,alice_smith,alice@example.com
2,jane_doe,jane@example.com
3,alice_smith,alice@example.com
4,jane_doe,jane@example.com
5,alice_smith,alice@example.com


### Drop a View

In [44]:
# Drop a view
cursor.execute("DROP VIEW user_emails")
conn.commit()